In [1]:
import os
import networkx as nx
import numpy as np
import pandas as pd

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from networkx.algorithms import node_classification

Dataset: wiki-talk datasets

In [2]:
# params
data_dir = 'data/wiki-talk/'
train_size = 0.5
lang = 'sk'    # specify which language of Wikipedia 

### Load edge info

In [3]:
edgelist = pd.read_csv(os.path.join(data_dir, f"{lang}-wiki-talk"), sep='\t', header=None, names=["target", "source", "timestamp"])
edgelist

,target,source,timestamp
0,8,8,2003-12-03T09:04:10Z
1,14,14,2004-02-23T09:10:57Z
2,16,14,2004-02-23T09:28:02Z
3,16,14,2004-02-25T20:41:16Z
4,28,14,2004-04-05T16:58:09Z
...,...,...,...
131879,66699,121620,2015-11-21T21:16:33Z
131880,66699,121883,2015-11-22T15:25:22Z
131881,39673,121883,2015-11-22T16:25:03Z
131882,12161,121883,2015-11-22T18:57:16Z


In [4]:
# We have to use an undirected graph here because the node classification algorithms in nx do not support directed graphs
G = nx.from_pandas_edgelist(edgelist)

### Load node info

In [5]:
df_nodes = pd.read_csv(os.path.join(data_dir, f"{lang}-user-group"), sep='\t', header=None)
df_nodes

,0,1
0,14940,1
1,36018,1
2,9422,0
3,74846,1
4,47824,1
...,...,...
148,12656,1
149,16434,1
150,4036,1
151,27162,2


In [6]:
# number of nodes
len(G.nodes)

41452

In [7]:
max_node_num = np.max(edgelist[["target", "source"]].values) + 1
node_ids = np.unique(edgelist[["target", "source"]].values)

In [8]:
# dataset (roles)
roles = df_nodes[[0,1]].values

y = [0] * max_node_num
for r in roles:
    y[r[0]] = r[1]

y = np.array([y[i] for i in node_ids])

In [9]:
df_nodes = pd.DataFrame({'id': node_ids, 'label': y})

In [10]:
df_nodes

,id,label
0,2,0
1,8,0
2,9,0
3,12,0
4,14,0
...,...,...
41447,121707,0
41448,121750,0
41449,121851,0
41450,121883,0


In [11]:
df_train, df_test = train_test_split(df_nodes, train_size = train_size)

In [12]:
node_ids_train, node_ids_test = df_train.iloc[:, 0].values, df_test.iloc[:, 0].values
labels_train, labels_test = df_train.iloc[:, -1].values.astype(str), df_test.iloc[:, -1].values.astype(str)
data_train = dict(zip(node_ids_train, labels_train))

In [13]:
nx.set_node_attributes(G, data_train, name="label")

### Node classification

Now we perform node classification using the built-in Harmonic Function method in NetworkX.

* Zhu, X., Ghahramani, Z., & Lafferty, J. (2003, August). Semi-supervised learning using gaussian fields and harmonic functions. In ICML (Vol. 3, pp. 912-919).

Note: this method is not scalable, so it might be problematic to run on large networks, such as the English Wiki-talk network `lang='en'`.

In [14]:
result = node_classification.harmonic_function(G)

The result contains the labels (predicted or ground truth) of all nodes. For example, the labels of the first five nodes:

In [15]:
result[:5]

['0', '0', '0', '0', '0']

To check the performance of the classification, we fetch the labels of the nodes in the test set.

In [16]:
dict_result = dict(zip(list(G), result))
labels_pred = [ dict_result.get(id) for id in node_ids_test ]

print('Accuracy (Harmonic Function): ', accuracy_score(labels_test, labels_pred))

Accuracy (Harmonic Function):  0.5913828042072758


Since the dataset is extremely imbalanced, i.e., a lot of nodes have label 0, representing normal users, accuracy is then not a good metric to evaluate the classifier. We calculate the recall score for each role.

In [17]:
recall_score(labels_test, labels_pred, average=None)

array([0.59269296, 0.16666667, 0.        ])

Similarly, we can perform node classification using the built-in Local and Global Consistency method.

* Zhou, D., Bousquet, O., Lal, T. N., Weston, J., & Schölkopf, B. (2004). Learning with local and global consistency. Advances in neural information processing systems, 16(16), 321-328.

In [18]:
result = node_classification.local_and_global_consistency(G)

In [19]:
dict_result = dict(zip(list(G), result))
labels_pred = [ dict_result.get(id) for id in node_ids_test ]

In [20]:
print('Accuracy (Local and Global Consistency): ', accuracy_score(labels_test, labels_pred))

Accuracy (Local and Global Consistency):  0.9970085882466467


Again a high accuracy does not mean we have a good role classifier, due to the highly imbalanced dataset.

In [21]:
recall_score(labels_test, labels_pred, average=None)

array([0.99995161, 0.        , 0.        ])